In [ ]:
## Calculate profiles

%reload_ext autoreload
%autoreload 2

from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import colormaps as cm
from matplotlib import colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from lib.Derived import Derived
from functions.plot_data import plot_data

In [ ]:
# Open data files
from functions.load_pickles import load_pickles
stokes_list, nova = load_pickles()

In [ ]:
# Extract each Stokes parameter into dictionary, to make it easier to work with
I = stokes_list['I']
Q = stokes_list['Q']
U = stokes_list['U']
V = stokes_list['V']

In [ ]:
# Select pixels of interest
plate_scale_x = 0.14857 # arcseconds per pixel
plate_scale_y = 0.16 # arcseconds per pixel
x_pix = np.array([500, 180, 290, 400, 320, 300])
y_pix = np.array([118, 113, 538, 354, 428, 413])
labels=['A', 'B', 'C', 'D', 'E', 'F']
# Plot polarization and selected points
# fig, ax, img = plot_data(nova.mp[:,:,0], colourmap='gist_earth', title='Selected pixels the polarisation map')
fig, ax, img = plot_data(nova.mp[:,:,0], colourmap='gist_earth')
fig.show()
ax.scatter(x_pix*plate_scale_x, y_pix*plate_scale_y, facecolors='none', edgecolors='red')
for i, label in enumerate(labels):
    ax.annotate(label, (x_pix[i]*plate_scale_x, y_pix[i]*plate_scale_y), textcoords="offset points", xytext=(0,10), ha='center', color='white')


In [ ]:
# Plot profiles of each pixel
line_wavelengths = [6301.5015, 6302.4893]
pixel_titles = [f'A (quiet Sun)', f'B (polarised Sun)', f'C (penumbra bright)', f'D (penmbra dark)', f'E (umbra dark)', f'F (umbra bridge)']
fig, axs = plt.subplots(3, 2, figsize=(16,12))
fig.suptitle('Stokes parameter variations over all wavelengths', fontsize=20)


# Choose a colormap
num_lines = 5
cmap = plt.get_cmap('nipy_spectral')
colors = [cmap(i / num_lines) for i in range(num_lines)]

for i in range(3):
    for j in range(2):
        k=1
        for param in stokes_list:
            axs[i, j].plot(stokes_list[param].wave_array, stokes_list[param].data_n[y_pix[i*2+j],x_pix[i*2+j],:], linestyle='-', label=param, color=colors[k])
            k=k+1
        axs[i, j].set_title(pixel_titles[i*2+j])
        axs[i, j].vlines(line_wavelengths, stokes_list['V'].data_n[y_pix[i*2+j],x_pix[i*2+j],:].min(), stokes_list['I'].data_n[y_pix[i*2+j],x_pix[i*2+j],:].max(), colors='gray', linestyles='dashed')


        # axs[i, j].legend()


wave_dip = 6302.045
axs[2, 0].vlines(wave_dip, stokes_list['I'].data_n[y_pix[4],x_pix[4],:].min(), stokes_list['I'].data_n[y_pix[4],x_pix[4],:].max(), colors='orange', linestyles='dashed')
axs[2, 1].vlines(wave_dip, stokes_list['I'].data_n[y_pix[5],x_pix[5],:].min(), stokes_list['I'].data_n[y_pix[5],x_pix[5],:].max(), colors='orange', linestyles='dashed')


wave_dip = 6302.02
axs[2, 0].vlines(wave_dip, stokes_list['V'].data_n[y_pix[4],x_pix[4],:].min(), stokes_list['V'].data_n[y_pix[4],x_pix[4],:].max(), colors='purple', linestyles='dashed')
axs[2, 1].vlines(wave_dip, stokes_list['V'].data_n[y_pix[5],x_pix[5],:].min(), stokes_list['V'].data_n[y_pix[5],x_pix[5],:].max(), colors='purple', linestyles='dashed')

axs[0, 0].legend()

In [ ]:
# Plot profiles of each pixel
line_wavelengths = [6301.5015, 6302.4893]
fig, axs = plt.subplots(3, 2, figsize=(11,7))
fig.suptitle('Stokes parameter variations over wavelength')


# Choose a colormap
num_lines = 5
cmap = plt.get_cmap('nipy_spectral')
colors = [cmap(i / num_lines) for i in range(num_lines)]

for i in range(3):
    for j in range(2):
        k=1
        for param in stokes_list:
            if param != 'I':
                axs[i, j].plot(stokes_list[param].wave_array, stokes_list[param].data_n[y_pix[i*2+j],x_pix[i*2+j],:], linestyle='-', label=param, color=colors[k])
            k=k+1

        axs[i, j].set_title(pixel_titles[i*2+j])
        axs[i, j].vlines(line_wavelengths, stokes_list['V'].data_n[y_pix[i*2+j],x_pix[i*2+j],:].min(), stokes_list['V'].data_n[y_pix[i*2+j],x_pix[i*2+j],:].max(), colors='gray', linestyles='dashed')


        # axs[i, j].legend()


# wave_dip = 6302.05
# axs[2, 0].vlines(wave_dip, stokes_list['I'].data_n[y_pix[4],x_pix[4],:].min(), stokes_list['I'].data_n[y_pix[4],x_pix[4],:].max(), colors='orange', linestyles='dashed')
# axs[2, 1].vlines(wave_dip, stokes_list['I'].data_n[y_pix[5],x_pix[5],:].min(), stokes_list['I'].data_n[y_pix[5],x_pix[5],:].max(), colors='orange', linestyles='dashed')


# wave_dip = 6302.02
# axs[2, 0].vlines(wave_dip, stokes_list['V'].data_n[y_pix[4],x_pix[4],:].min(), stokes_list['V'].data_n[y_pix[4],x_pix[4],:].max(), colors='purple', linestyles='dashed')
# axs[2, 1].vlines(wave_dip, stokes_list['V'].data_n[y_pix[5],x_pix[5],:].min(), stokes_list['V'].data_n[y_pix[5],x_pix[5],:].max(), colors='purple', linestyles='dashed')

axs[0, 0].legend()
plt.tight_layout()

In [ ]:
# Plot profiles of each pixel
fig, axs = plt.subplots(4, 1, figsize=(6, 10), sharex=True)
# fig.suptitle('Parameter variation for selected pixels', fontsize=20)

# Choose a colormap
num_lines = 6
cmap = plt.get_cmap('turbo')
colors = [cmap(i / num_lines) for i in range(num_lines)]
ylabel=[r'$ I\:/\:I_{0,QS}$', r'$ Q\:/\:I_{0,QS}$', r'$ U\:/\:I_{0,QS}$', r'$ V\:/\:I_{0,QS}$']

labels=['A', 'B', 'C', 'D', 'E', 'F']

for i, param in enumerate(stokes_list):
    ax = axs[i]
    for j in range(num_lines):
        ax.plot(stokes_list[param].wave_array, stokes_list[param].data_n[y_pix[j],x_pix[j],:], linestyle='-', label=labels[j], color=colors[j], linewidth=1)
    # ax.set_title(param)
    ax.set_ylabel(ylabel[i])
    if i == 3:
        ax.set_xlabel(r'Wavelength ($\AA$)')
    else:
        ax.set_xlabel('')
    ax.legend(loc='lower right')

fig.tight_layout()

In [ ]:
# Plot profiles of each pixel
fig, axs = plt.subplots(2, 2, figsize=(8,5))
# fig.suptitle('Parameter variation for selected pixels', fontsize=20)

# Choose a colormap
num_lines = 6
cmap = plt.get_cmap('turbo')
colors = [cmap(i / num_lines) for i in range(num_lines)]
ylabel=[r'$ I\:/\:I_{0,QS}$', r'$ Q\:/\:I_{0,QS}$', r'$ U\:/\:I_{0,QS}$', r'$ V\:/\:I_{0,QS}$']

labels=['A', 'B', 'C', 'D', 'E', 'F']

i=0
for param in stokes_list:
    row, col = divmod(i, 2)
    for j in range(num_lines):
        axs[row, col].plot(stokes_list[param].wave_array, stokes_list[param].data_n[y_pix[j],x_pix[j],:], linestyle='-', label=labels[j], color=colors[j], linewidth=0.8)
        # axs[row, col].set_title(param)
        axs[row, col].set_ylabel(ylabel[i])

        if row == 1:
            axs[row, col].set_xlabel(r'Wavelength ($\AA$)')
        else:
            axs[row, col].set_xlabel('')

        # axs[row, col].vlines(line_wavelengths, stokes_list[param].data_n[y_pix[j],x_pix[j],:].min(), stokes_list[param].data_n[y_pix[j],x_pix[j],:].max(), colors='gray', linestyles='dashed')
        # axs[i, j].legend()

    i += 1

axs[1, 0].legend()
fig.tight_layout()


In [ ]:
# Plot profiles of each pixel
fig, axs = plt.subplots(figsize=(5, 3))
# fig.suptitle('Parameter variation for selected pixels', fontsize=20)

# Choose a colormap
num_lines = 6
cmap = plt.get_cmap('turbo')
colors = [cmap(i / num_lines) for i in range(num_lines)]
ylabel=[r'$ I\:/\:I_{0,QS}$', r'$ Q\:/\:I_{0,QS}$', r'$ U\:/\:I_{0,QS}$', r'$ V\:/\:I_{0,QS}$']

labels=['A', 'B', 'C', 'D', 'E', 'F']

i=0
row, col = divmod(i, 2)
for j in range(num_lines):
    if (j<4):
        pass
    else:
        axs.plot(stokes_list['V'].wave_array[60:90], stokes_list['V'].data_n[y_pix[j],x_pix[j],60:90:], linestyle='-', label=labels[j], color=colors[j], linewidth=0.8)
    # axs[row, col].set_title(param)
    axs.set_ylabel(ylabel[3])

    axs.set_xlabel(r'Wavelength ($\AA$)')

    # axs[row, col].vlines(line_wavelengths, stokes_list[param].data_n[y_pix[j],x_pix[j],:].min(), stokes_list[param].data_n[y_pix[j],x_pix[j],:].max(), colors='gray', linestyles='dashed')
    # axs[i, j].legend()

    i += 1

wave_dip = 6302.49
axs.vlines(wave_dip, stokes_list['V'].data_n[y_pix[5],x_pix[5],:].min(), stokes_list['V'].data_n[y_pix[5],x_pix[5],:].max(), colors='purple', linestyles='dashed', linewidth=0.8)

axs.legend()
fig.tight_layout()

In [ ]:
for i in range(6):
    print(f'{I.data_n[y_pix[i],x_pix[i],:5].mean():.4f} & {Q.data_n[y_pix[i],x_pix[i],:5].mean():.4f} & {U.data_n[y_pix[i],x_pix[i],:5].mean():.4f} & {V.data_n[y_pix[i],x_pix[i],:5].mean():.4f}')
        # print(f'{I.data_n[x_pix[i],y_pix[i],:5].mean():.4f} & {Q.data_n[x_pix[i],y_pix[i],:5].mean():.4f} & {U.data_n[x_pix[i],y_pix[i],:5].mean():.4f} & {V.data_n[x_pix[i],y_pix[i],:5].mean():.4f}')

In [ ]:
# Plot profiles of each pixel
pixel_titles = [f'Quiet sun', f'Exterior to  sunspot', f'Penumbra 1', f'Penmbra 2', f'Umbra 1', f'Umbra 2']
fig, axs = plt.subplots(2, 2, figsize=(8,6))
fig.suptitle('Profile variation for selected pixels', fontsize=20)

# Choose a colormap
num_lines = 6
cmap = plt.get_cmap('turbo')
colors = [cmap(i / num_lines) for i in range(num_lines)]

i=0
for param in stokes_list:
    row, col = divmod(i, 2)
    for j in range(num_lines):
        if (j<4):
            pass
        else:
            axs[row, col].plot(stokes_list[param].wave_array, stokes_list[param].data_n[y_pix[j],x_pix[j],:], linestyle='-', label=pixel_titles[j], color=colors[j])
            axs[row, col].set_title(param)
            # axs[row, col].vlines(line_wavelengths, stokes_list[param].data_n[y_pix[j],x_pix[j],:].min(), stokes_list[param].data_n[y_pix[j],x_pix[j],:].max(), colors='gray', linestyles='dashed')
            # axs[i, j].legend()

    i += 1


axs[1, 0].legend()
